### Initial setup

In [1]:
import cv2
import os
import json
from utils import *

images_dir = './samples'
input = 'input.json'
output = 'output.json'

### Background median testing

In [2]:
# ----------------------------------------------

with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
image_index = 0

image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))
image = cv2.resize(image, (0, 0), fx = 0.15, fy = 0.15)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# ----------------------------------------------

def process_image():
    bg_median_mask = background_median(hsv, hue_tolerance, saturation_tolerance, value_tolerance, (5, 5))
    bg_median = cv2.bitwise_and(image, image, mask=bg_median_mask)
    contours, _ = cv2.findContours(bg_median_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = filter_contours(contours)
    image_copy = image.copy()
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(image_copy, (x, y), (x + w, y + h), (255, 255, 0), 2)
    combined_image = np.hstack((bg_median, image_copy))
    cv2.imshow(window_name, combined_image)

def update_hue_tolerance(*args):
    global hue_tolerance
    hue_tolerance = cv2.getTrackbarPos("Hue", window_name)
    process_image()

def update_saturation_tolerance(*args):
    global saturation_tolerance
    saturation_tolerance = cv2.getTrackbarPos("Saturation", window_name)
    process_image()

def update_value_tolerance(*args):
    global value_tolerance
    value_tolerance = cv2.getTrackbarPos("Value", window_name)
    process_image()

# ----------------------------------------------

hue_tolerance = 100
saturation_tolerance = 100
value_tolerance = 100

window_name = "TEST"
cv2.namedWindow(window_name)

cv2.createTrackbar("Hue", window_name, hue_tolerance, 180, update_hue_tolerance)
cv2.createTrackbar("Saturation", window_name, saturation_tolerance, 255, update_saturation_tolerance)
cv2.createTrackbar("Value", window_name, value_tolerance, 255, update_value_tolerance)

process_image()

cv2.waitKey(0)
cv2.destroyAllWindows()

# ----------------------------------------------

### Canny testing

In [3]:
# ----------------------------------------------

with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
image_index = 0

image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))
image = cv2.resize(image, (0, 0), fx = 0.15, fy = 0.15)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# ----------------------------------------------

def process_image():
    canny_mask = canny_edges(gray, (3, 3), low_threshold, high_threshold)
    canny = cv2.bitwise_and(image, image, mask=canny_mask)
    contours, _ = cv2.findContours(canny_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = filter_contours(contours)
    image_copy = image.copy()
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(image_copy, (x, y), (x + w, y + h), (255, 255, 0), 2)
    combined_image = np.hstack((canny, image_copy))
    cv2.imshow(window_name, combined_image)

def update_low_threshold(*args):
    global low_threshold
    low_threshold = cv2.getTrackbarPos("Low", window_name)
    process_image()

def update_high_threshold(*args):
    global high_threshold
    high_threshold = cv2.getTrackbarPos("High", window_name)
    process_image()

# ----------------------------------------------

low_threshold = 100
high_threshold = 200

window_name = "TEST"
cv2.namedWindow(window_name, cv2.WINDOW_AUTOSIZE)

cv2.createTrackbar("Low", window_name, low_threshold, 255, update_low_threshold)
cv2.createTrackbar("High", window_name, high_threshold, 255, update_high_threshold)

process_image()

cv2.waitKey(0)
cv2.destroyAllWindows()

# ----------------------------------------------